In [1]:
#installation
# !conda install pytorch torchvision cudatoolkit=10.2 -c pytorch
# !pip install transformers
# !pip install pytorch_transformers

In [2]:
# Tutorial copied from https://github.com/huggingface/transformers#usage

In [3]:
import torch
from transformers import BertModel, BertTokenizer

# Transformers has a unified API
# for 10 transformer architectures and 30 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut
MODELS = [(BertModel,       BertTokenizer,       'bert-base-uncased')
         ]


model_class = BertModel
tokenizer_class = BertTokenizer
pretrained_weights= 'bert-base-uncased'
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  
# Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
with torch.no_grad():
    last_hidden_states = model(input_ids)[0]  
    # Models outputs are now tuples

tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

# Models can return full list of hidden-states & attentions weights at each layer
model = model_class.from_pretrained(pretrained_weights,
                                    output_hidden_states=True,
                                    output_attentions=True)
input_ids = torch.tensor([tokenizer.encode("Let's see all hidden-states and attentions on this text")])
all_hidden_states, all_attentions = model(input_ids)[-2:]

# # Models are compatible with Torchscript
# model = model_class.from_pretrained(pretrained_weights, torchscript=True)
# traced_model = torch.jit.trace(model, (input_ids,))

# # Simple serialization for models and tokenizers
# model.save_pretrained('./directory/to/save/')  # save
# model = model_class.from_pretrained('./directory/to/save/')  # re-load
# tokenizer.save_pretrained('./directory/to/save/')  # save
# tokenizer = BertTokenizer.from_pretrained('./directory/to/save/')  # re-load

# # SOTA examples for GLUE, SQUAD, text generation...

In [4]:
len(all_hidden_states)

13